In [100]:
import re
import os
import json
import nltk
import math
import random
import unicodedata

import pickle
import numpy as np
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [101]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalize(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s
df = open('context_responses.json').read()
intents = json.loads(df)

words = []
classes = []
documents = []


for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        pattern = normalize(pattern)
        word_list = pattern.split(" ")
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        
        if(intent['tag'] not in classes):
            classes.append(intent['tag'])

classes = sorted(list(set(classes)))
print("classes:", classes)
print(documents)

classes: ['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']
[(['hi'], 'greeting'), (['hey'], 'greeting'), (['how', 'are', 'you'], 'greeting'), (['is', 'anyone', 'there'], 'greeting'), (['hello'], 'greeting'), (['good', 'day'], 'greeting'), (['bye'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['goodbye'], 'goodbye'), (['thanks'], 'thanks'), (['thank', 'you'], 'thanks'), (['that', 's', 'helpful'], 'thanks'), (['thank', 's', 'a', 'lot'], 'thanks'), (['which', 'items', 'do', 'you', 'have'], 'items'), (['what', 'kinds', 'of', 'items', 'are', 'there'], 'items'), (['what', 'do', 'you', 'sell'], 'items'), (['do', 'you', 'take', 'credit', 'cards'], 'payments'), (['do', 'you', 'accept', 'mastercard'], 'payments'), (['can', 'i', 'pay', 'with', 'paypal'], 'payments'), (['are', 'you', 'cash', 'only'], 'payments'), (['how', 'long', 'does', 'delivery', 'take'], 'delivery'), (['how', 'long', 'does', 'shipping', 'take'], 'delivery'), (['when', 'do', 'i', 'get', 'my', 'd

In [102]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words]

pickle.dump(words,open('words.pkl', 'wb'))
pickle.dump(classes,open('classes.pkl', 'wb'))

print('fin')

fin


In [103]:
import random

training = []

#output array
out = [0]*len(classes)

for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        if word in word_patterns:
            bag.append(1)
        else:
            bag.append(0)
    
    out_row = list(out)
    out_row[classes.index(doc[1])] = 1
    training.append([bag, out_row])


random.shuffle(training)
#training =  torch.FloatTensor(training)
print()
training = np.array(training, dtype='object')
# create training and testing lists. X - patterns, Y - intents
train_x = np.array(list(training[:,0]))
train_y = np.array(list(training[:,1]))


#training = np.array(training, dtype='int64')
#train_ds = torch.utils.data.DataLoader(training, batch_size = 5, shuffle=True, num_workers=2)
print("done")


done


In [120]:
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(len(training[:,0][0]), 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64,len(training[:,1][0]))
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.dropout(x)        
        x = F.relu(self.fc2(x))
        x = F.dropout(x)
        x = F.softmax(self.fc3(x), dim=0)
        return x

NS_Model = nn.Sequential(
    nn.Linear(len(training[:,0][0]), 128),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(64,len(training[:,1][0])),
    nn.Softmax()
).cuda()
print(NS_Model)

if device == "cuda":
    model = Net().cuda()
else:
    model = Net()
    
print(model)

#add sequential API

Sequential(
  (0): Linear(in_features=88, out_features=128, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=128, out_features=64, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=64, out_features=7, bias=True)
  (7): Softmax(dim=None)
)
Net(
  (fc1): Linear(in_features=88, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=7, bias=True)
)


In [121]:

epochs = 200
optimizer = optim.SGD(NS_Model.parameters(), lr=0.01, momentum=0.9)
#criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss(reduction='mean')   
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(training, 0):
        inputs, labels = data
        
        labels = torch.FloatTensor(labels).to(device)
        inputs = torch.FloatTensor(inputs).to(device)

        optimizer.zero_grad()
        
        outputs = NS_Model(inputs)
               
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    outputs = (outputs>0.5).float()
    correct = (outputs == labels).float().sum()
    print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1,epochs, loss.data, correct/outputs.shape[0]))
    
        
print("Fin Train")

Epoch 1/200, Loss: 0.438, Accuracy: 0.857
Epoch 2/200, Loss: 0.425, Accuracy: 0.857
Epoch 3/200, Loss: 0.421, Accuracy: 0.857
Epoch 4/200, Loss: 0.416, Accuracy: 0.857
Epoch 5/200, Loss: 0.418, Accuracy: 0.857
Epoch 6/200, Loss: 0.408, Accuracy: 0.857
Epoch 7/200, Loss: 0.400, Accuracy: 0.857
Epoch 8/200, Loss: 0.407, Accuracy: 0.857
Epoch 9/200, Loss: 0.373, Accuracy: 0.857
Epoch 10/200, Loss: 0.392, Accuracy: 0.857
Epoch 11/200, Loss: 0.407, Accuracy: 0.857
Epoch 12/200, Loss: 0.419, Accuracy: 0.857
Epoch 13/200, Loss: 0.387, Accuracy: 0.857
Epoch 14/200, Loss: 0.418, Accuracy: 0.857
Epoch 15/200, Loss: 0.377, Accuracy: 0.857
Epoch 16/200, Loss: 0.367, Accuracy: 0.857
Epoch 17/200, Loss: 0.359, Accuracy: 0.857
Epoch 18/200, Loss: 0.343, Accuracy: 0.857
Epoch 19/200, Loss: 0.309, Accuracy: 0.857
Epoch 20/200, Loss: 0.234, Accuracy: 0.857
Epoch 21/200, Loss: 0.283, Accuracy: 0.857
Epoch 22/200, Loss: 0.345, Accuracy: 0.857
Epoch 23/200, Loss: 0.343, Accuracy: 0.857
Epoch 24/200, Loss: 

Epoch 199/200, Loss: 0.000, Accuracy: 1.000
Epoch 200/200, Loss: 0.000, Accuracy: 1.000
Fin Train


In [122]:
epochs = 200
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss(reduction='mean')   
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(training, 0):
        inputs, labels = data
        
        labels = torch.FloatTensor(labels).to(device)
        inputs = torch.FloatTensor(inputs).to(device)

        optimizer.zero_grad()
        
        outputs = model(inputs)
               
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    outputs = (outputs>0.5).float()
    correct = (outputs == labels).float().sum()
    print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1,epochs, loss.data, correct/outputs.shape[0]))
    
        
print("Fin Train")

Epoch 1/200, Loss: 0.417, Accuracy: 0.857
Epoch 2/200, Loss: 0.420, Accuracy: 0.857
Epoch 3/200, Loss: 0.414, Accuracy: 0.857
Epoch 4/200, Loss: 0.406, Accuracy: 0.857
Epoch 5/200, Loss: 0.439, Accuracy: 0.857
Epoch 6/200, Loss: 0.408, Accuracy: 0.857
Epoch 7/200, Loss: 0.402, Accuracy: 0.857
Epoch 8/200, Loss: 0.402, Accuracy: 0.857
Epoch 9/200, Loss: 0.409, Accuracy: 0.857
Epoch 10/200, Loss: 0.412, Accuracy: 0.857
Epoch 11/200, Loss: 0.416, Accuracy: 0.857
Epoch 12/200, Loss: 0.379, Accuracy: 0.857
Epoch 13/200, Loss: 0.396, Accuracy: 0.857
Epoch 14/200, Loss: 0.360, Accuracy: 0.857
Epoch 15/200, Loss: 0.383, Accuracy: 0.857
Epoch 16/200, Loss: 0.376, Accuracy: 0.857
Epoch 17/200, Loss: 0.382, Accuracy: 0.857
Epoch 18/200, Loss: 0.381, Accuracy: 0.857
Epoch 19/200, Loss: 0.378, Accuracy: 0.857
Epoch 20/200, Loss: 0.352, Accuracy: 0.857
Epoch 21/200, Loss: 0.346, Accuracy: 0.857
Epoch 22/200, Loss: 0.248, Accuracy: 0.857
Epoch 23/200, Loss: 0.255, Accuracy: 0.857
Epoch 24/200, Loss: 

Epoch 195/200, Loss: 0.000, Accuracy: 1.000
Epoch 196/200, Loss: 0.000, Accuracy: 1.000
Epoch 197/200, Loss: 0.000, Accuracy: 1.000
Epoch 198/200, Loss: 0.000, Accuracy: 1.000
Epoch 199/200, Loss: 0.000, Accuracy: 1.000
Epoch 200/200, Loss: 0.000, Accuracy: 1.000
Fin Train


In [93]:
torch.save(model.state_dict(), 'model.pth');
print("fin")

fin


In [94]:
df = 'model.pth'
model = torch.load(df)

In [98]:
intents = json.loads(open('context_responses.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalize(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

def clean_(sentence):
    wrds= sentence.split(' ');
    swrds = [lemmatizer.lemmatize(wrd.lower()) for wrd in wrds]
    return swrds


def conv(sentence, words, show_details=True):
    sentence_words = clean_(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % word)
    return(np.array(bag))


def getResponse(tag):
    for i in range(len(intents['intents'])):
        if(intents['intents'][i]['tag'] == tag):
            responses = intents['intents'][i]['responses']
            response = random.choice(responses)
            return response

def predict_class(sentence):
    p = conv(sentence, words,show_details=False)

    model = Net().cuda()
    model.load_state_dict(torch.load("model.pth"))
    model.eval()
    model.train()
    
    with torch.no_grad():
        res = model(torch.FloatTensor(p).to(device))
    
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

print("---Type Exit to Exit Debug to Debug ---- ")
ins = "Filler"
stats_chart = []

while True:
    ins = input("User> ")
    if ins == "Debug":
        if(len(stats_chart) == 0):
            print("Nothing To Show")
            continue
        print(stats_chart)
        ins  = "Filler"
        continue
        
    if ins == "Exit":
        break

    ins = normalize(ins)
    stats = predict_class(ins)
    stats_chart.append(stats)
    print("Bot> ",getResponse(stats[0]['intent']))
    
    

---Type Exit to Exit Debug to Debug ---- 
User> Hello
Bot>  Hello, thanks for visiting
User> I want to buy something
Bot>  We accept most major credit cards, and Paypal
User> What do you have for sale?
Bot>  We sell coffee and tea
User> ok
Bot>  Hi there, what can I do for you?
User> thanks
Bot>  Happy to help!
User> Exit
